In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backpor

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-04-03 21:56:03--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.1s    

2022-04-03 21:56:03 (9.70 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

###Load Amazon Data into Spark DataFrame

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Toys_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

# Read in the Review dataset as a DataFrame
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   18778586| RDIJS7QYB6XNR|B00EDBY7X8|     122952789|Monopoly Junior B...|            Toys|          5|            0|          0|   N|                Y|          Five Stars|        Excellent!!!| 2015-08-31|
|         US|   24769659|R36ED1U38IELG8|B00D7JFOPC|     952062646|56 Pieces of Wood...|            Toys|          5|    

In [5]:
# Create the vine_table. DataFrame
vine_df = df.select(['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase'])
vine_df.show()
vine_df.describe().show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RDIJS7QYB6XNR|          5|            0|          0|   N|                Y|
|R36ED1U38IELG8|          5|            0|          0|   N|                Y|
| R1UE3RPRGCOLD|          2|            1|          1|   N|                Y|
|R298788GS6I901|          5|            0|          0|   N|                Y|
|  RNX4EXOBBPN5|          1|            1|          1|   N|                Y|
|R3BPETL222LMIM|          5|            0|          0|   N|                Y|
|R3SORMPJZO3F2J|          3|            2|          2|   N|                Y|
|R2RDOJQ0WBZCF6|          5|            0|          0|   N|                Y|
|R2B8VBEPB4YEZ7|          5|            0|          0|   N|                Y|
|R1CB783I7B0U52|          1|            0|          1|   N|     

In [6]:
# Step 1
# Filter the data and create a new DataFrame or table to retrieve all the rows where the total_votes count is equal to or greater than 20
# to pick reviews that are more likely to be helpful and to avoid having division by zero errors later on.
votes_vine_df = vine_df.filter("total_votes>=20")
votes_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| ROP6ITXO8K5V2|          5|           23|         27|   N|                Y|
|R3ND1LVU7AXCVF|          1|           21|         21|   N|                Y|
| R9I5FOLKU99RY|          5|           19|         20|   Y|                N|
|R1QS8AOD6HX3ED|          4|           59|         81|   N|                N|
|R3ED60RC69CCQ6|          5|           22|         23|   Y|                N|
|R2MDI1TP46VSYD|          3|            9|         21|   N|                Y|
|R2JM687C525WR9|          3|           33|         33|   Y|                N|
|R2LWX4TZ67FWPT|          4|           50|         50|   N|                N|
|R29IYHPYD14AGI|          3|           84|         84|   N|                Y|
|R1F2I723WRK5QV|          5|           20|         20|   N|     

In [7]:
# Step 2
# Filter the new DataFrame or table created in Step 1 and create a new DataFrame or table to retrieve all the rows
# where the number of helpful_votes divided by total_votes is equal to or greater than 50%.
percent_vine_df = votes_vine_df.filter("helpful_votes/total_votes>=0.5")
percent_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| ROP6ITXO8K5V2|          5|           23|         27|   N|                Y|
|R3ND1LVU7AXCVF|          1|           21|         21|   N|                Y|
| R9I5FOLKU99RY|          5|           19|         20|   Y|                N|
|R1QS8AOD6HX3ED|          4|           59|         81|   N|                N|
|R3ED60RC69CCQ6|          5|           22|         23|   Y|                N|
|R2JM687C525WR9|          3|           33|         33|   Y|                N|
|R2LWX4TZ67FWPT|          4|           50|         50|   N|                N|
|R29IYHPYD14AGI|          3|           84|         84|   N|                Y|
|R1F2I723WRK5QV|          5|           20|         20|   N|                Y|
| RRO8C1IVZMD6H|          5|           22|         22|   N|     

In [8]:
# Step 3
# Filter the DataFrame or table created in Step 2, and create a new DataFrame or table that retrieves
# all the rows where a review was written as part of the Vine program (paid), vine == 'Y'.
paid_vine_df = percent_vine_df.filter("vine=='Y'")
paid_vine_df.show()
paid_vine_df.describe().show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R9I5FOLKU99RY|          5|           19|         20|   Y|                N|
|R3ED60RC69CCQ6|          5|           22|         23|   Y|                N|
|R2JM687C525WR9|          3|           33|         33|   Y|                N|
| RUB4AUWGHG16G|          4|           25|         27|   Y|                N|
|R1S3IDYJUA6V2G|          5|           28|         32|   Y|                N|
| RZFLN8FD5Y2M6|          4|           25|         28|   Y|                N|
|R1V8NDXD9SETO5|          3|          320|        352|   Y|                N|
|R2H4VEFWHP50M3|          5|           26|         36|   Y|                N|
|R3DP6Y0A8WXUOV|          5|           21|         21|   Y|                N|
| R49L85GZIKH4D|          3|           33|         36|   Y|     

In [9]:
# Step 4
# Repeat Step 3, but this time retrieve all the rows where the review was not part of the Vine program (unpaid), vine == 'N'.
unpaid_vine_df = percent_vine_df.filter("vine=='N'")
unpaid_vine_df.show()
unpaid_vine_df.describe().show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| ROP6ITXO8K5V2|          5|           23|         27|   N|                Y|
|R3ND1LVU7AXCVF|          1|           21|         21|   N|                Y|
|R1QS8AOD6HX3ED|          4|           59|         81|   N|                N|
|R2LWX4TZ67FWPT|          4|           50|         50|   N|                N|
|R29IYHPYD14AGI|          3|           84|         84|   N|                Y|
|R1F2I723WRK5QV|          5|           20|         20|   N|                Y|
| RRO8C1IVZMD6H|          5|           22|         22|   N|                N|
|R2GHOWIWDVLN18|          1|           28|         36|   N|                Y|
|R30PJ9XUHFTY5D|          5|           42|         43|   N|                N|
|R2F7D9N3SSQC20|          4|           32|         34|   N|     

In [10]:
# Step 5
# Determine the total number of reviews, the number of 5-star reviews, and the percentage of 5-star reviews for the two types of review (paid vs unpaid).
total_reviews = vine_df.count()
five_star_reviews = vine_df.select('review_id').filter('star_rating=5').count()
five_star_percentage_paid = (paid_vine_df.select('review_id').filter('star_rating=5').count() / paid_vine_df.count())*100
five_star_percentage_unpaid = (unpaid_vine_df.select('review_id').filter('star_rating=5').count() / unpaid_vine_df.count())*100

print(f"1. Total number of reviews is: {total_reviews:,};")
print(f"2. Number of 5-star reviews is: {five_star_reviews:,};")
print(f"3. Percentage of paid 5-star reviews: {five_star_percentage_paid:.1f}%;")
print(f"4. Percentage of unpaid 5-star reviews: {five_star_percentage_unpaid:.1f}%.")

1. Total number of reviews is: 4,864,249;
2. Number of 5-star reviews is: 3,076,917;
3. Percentage of paid 5-star reviews: 34.1%;
4. Percentage of unpaid 5-star reviews: 48.3%.


In [11]:
# Vine reviews and non-Vine reviews
print(f"There were {paid_vine_df.count():,} Vine reviews and {unpaid_vine_df.count():,} non-Vine reviews.")

There were 1,266 Vine reviews and 62,028 non-Vine reviews.


In [12]:
# 5-star Vine and non-Vine reviews
print(f"There were {paid_vine_df.select('review_id').filter('star_rating=5').count():,} 5-star Vine reviews and {unpaid_vine_df.select('review_id').filter('star_rating=5').count():,} 5-star non-Vine reviews.")

There were 432 5-star Vine reviews and 29,982 5-star non-Vine reviews.


In [25]:
# Non-verified purchases - Vine / non-Vine:
non_verified_paid = paid_vine_df.select("review_id").filter("verified_purchase=='N'").count()
non_verified_unpaid = unpaid_vine_df.select("review_id").filter("verified_purchase=='N'").count()
percent_non_ver_paid = (non_verified_paid / paid_vine_df.count())*100
percent_non_ver_unpaid = (non_verified_unpaid / unpaid_vine_df.count())*100

print(f"Total of non-verified purchases (Vine): {non_verified_paid:,}")
print(f"Total of non-verified purchases (non-Vine): {non_verified_unpaid:,}")
print(f"Percent of non-verified purchases (Vine): {percent_non_ver_paid:.1f}%")
print(f"Percent of non-verified purchases (non-Vine): {percent_non_ver_unpaid:.1f}%")

Total of non-verified purchases (Vine): 1,262
Total of non-verified purchases (non-Vine): 32,449
Percent of non-verified purchases (Vine): 99.7%
Percent of non-verified purchases (non-Vine): 52.3%


In [26]:
# 5-star reviews and non-verified purchases (Vine and non-Vine members):
five_star_paid_non_verified = paid_vine_df.select("review_id").filter("verified_purchase=='N'").filter("star_rating=5").count()
five_star_unpaid_non_verified = unpaid_vine_df.select("review_id").filter("verified_purchase=='N'").filter("star_rating=5").count()
percent_five_star_paid_nv = (five_star_paid_non_verified / paid_vine_df.count())*100
percent_five_star_unpaid_nv = (five_star_unpaid_non_verified / unpaid_vine_df.count())*100

print(f"5-star non-verified Vine: {five_star_paid_non_verified:,}")
print(f"5-star non-verified non-Vine: {five_star_unpaid_non_verified:,}")
print(f"Percent of 5-star reviews from Vine members with unverified purchase: {percent_five_star_paid_nv:.1f}%")
print(f"Percent of 5-star reviews from non-Vine members with unverified purchase: {percent_five_star_unpaid_nv:.1f}%")

5-star non-verified Vine: 430
5-star non-verified non-Vine: 15,660
Percent of 5-star reviews from Vine members with unverified purchase: 34.0%
Percent of 5-star reviews from non-Vine members with unverified purchase: 25.2%
